# Installation

In [ ]:
# remove `!` if running the line in a terminal
!pip install -U RelevanceAI[notebook]==1.2.7


# Setup

First, you need to set up a client object to interact with RelevanceAI.

In [ ]:
from relevanceai import Client

"""
You can sign up/login and find your credentials here: https://cloud.relevance.ai/sdk/api
Once you have signed up, click on the value under `Activation token` and paste it here
"""
client = Client()



# Data

You will need to have a dataset under your Relevance AI account. You can either use our e-commerce dataset as shown below or follow the tutorial on how to create your own dataset.

Our e-commerce dataset includes fields such as `product_title`, as well as the vectorized version of the field `product_title_clip_vector_`. Loading these documents can be done via:



## Load the data

In [ ]:
from relevanceai.datasets import get_ecommerce_dataset_encoded

documents = get_ecommerce_dataset_encoded()
{k:v for k, v in documents[0].items() if '_vector_' not in k}


## Upload the data to Relevance AI

Run the following cell, to upload these documents into your personal Relevance AI account under the name `quickstart_clustering_kmeans`

In [ ]:
df = client.Dataset("quickstart_kmeans_clustering")
df.insert_documents(documents)


## Check the data

In [ ]:
df.health


In [ ]:
df.schema


# Clustering

We apply the Kmeams clustering algorithm to the vector field, `product_title_clip_vector_`

In [6]:
from relevanceai.clusterer import KMeansModel

VECTOR_FIELD = "product_title_clip_vector_"
KMEAN_NUMBER_OF_CLUSTERS = 5
ALIAS = "kmeans_" + str(KMEAN_NUMBER_OF_CLUSTERS)

model = KMeansModel(k=KMEAN_NUMBER_OF_CLUSTERS)
clusterer = client.ClusterOps(alias=ALIAS, model=model)


In [7]:
clustered_docs = clusterer.fit_predict_documents(
        vector_fields=['product_title_clip_vector_'],
        documents=documents, inplace=False
    )


In [9]:
# Writing back the clustering results to the dataset
df.upsert_documents(documents=clustered_docs)


{'failed_documents': [], 'failed_documents_detailed': [], 'inserted': 739}

In [ ]:
# Get the centroid's vector and insert them as centroids into Relevance AI
centroids = clusterer.get_centroid_documents()
clusterer.insert_centroid_documents(centroids, df)


In [ ]:
# Get the documents that are closest to the center of each cluster
clusterer.list_closest_to_center(df)


We download a small sample and show the clustering results using our json_shower.

In [ ]:
from relevanceai import show_json

sample_documents = df.sample(n=5)
samples = [{
    'product_title':d['product_title'],
    'cluster':d['_cluster_'][VECTOR_FIELD][ALIAS]
} for d in sample_documents]

show_json(samples, text_fields=['product_title', 'cluster'])

